<a href="https://colab.research.google.com/github/anadiedrichs/2023-vision-computacional/blob/main/TP2_vision_Diedrichs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP2 - VISION COMPUTACIONAL

 Ana Laura Diedrichs
 SIU a0902
 9Co2022

## Ejercicio 1

Implementar la función ***create_gauss_filter(h, w, k_size, sigma)*** para crear filtros gaussianos para filtrado espectral. Debe retornar un filtro gaussiano de tamaño HxW en dominio espacial y su transformada de Fourier.

1.  Graficar ambas representaciones para diferentes tamaños de kernel y sigma. Aplicar el filtro una imagen para validar el funcionamiento en el dominio espectral.


In [ ]:
#Si queremos que las imágenes sean mostradas en una ventana emergente quitar el inline
%matplotlib inline
#%matplotlib qt
# OpenCV-Python utiliza NumPy para el manejo de imágenes
import numpy as np
# cv2 es el módulo python para acceder a OpenCV
import cv2 as cv
# matplotlib para mostrar imágenes, perfiles, histogramas, etc
import matplotlib.pyplot as plt
# para importar imagenes desde urls
from skimage import io
# mostrar imagenes opencv en googlecolab
from google.colab.patches import cv2_imshow

In [ ]:
#https://www.programcreek.com/python/example/89346/cv2.getGaussianKernel

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

def create_gauss_filter(h, w, k_size, sigma):

    # k_size debe ser impar
    assert k_size > 0 and k_size % 2 != 0

    # h y w positivos y TODO mayor al kernel
    assert h > 0 and w > 0

    # Crear un kernel gaussiano en el dominio espacial
    kernel = cv2.getGaussianKernel(k_size, sigma)
    #print(kernel)
    # creo la matriz kernel de tamaño k_size x k_size
    kernel = np.outer(kernel, kernel)

    # normalizar
    kernel = kernel / np.sum(kernel)
    assert np.sum(kernel) == 1

    #  rellenar el resto con cero.
    img_1 = np.zeros([h,w])
    # calculo el centro de la imagen
    nb = img_1.shape[0]
    na = kernel.shape[0]
    lower = (nb // 2) - (na // 2)
    upper = (nb // 2) + (na // 2)
    # centrar el kernel en matrix(h,w),
    img_1[lower:upper+1, lower:upper+1] = kernel

    # Calcular la transformada de Fourier del kernel
    kernel_fft = np.fft.fft2(img_1)

    kernel_fft_shift = np.fft.fftshift(kernel_fft)

    return img_1, kernel_fft_shift


# Parámetros de prueba
height = 256
width = 256
kernel_size = 15
sigma_value = 5

# Crear filtro gaussiano y su transformada de Fourier
gauss_filter, gauss_filter_fft = create_gauss_filter(height, width, kernel_size, sigma_value)

print(gauss_filter[121:136, 121:136])

# Visualizar el filtro gaussiano en el dominio espacial
plt.subplot(121), plt.imshow(gauss_filter, cmap='gray')
plt.title('Filtro Gaussiano (Dominio Espacial)'), plt.xticks([]), plt.yticks([])

plt.show()

# Visualizar la transformada de Fourier del filtro gaussiano
out = 20*np.log(np.abs(gauss_filter_fft)) # para graficar, el plot.
plt.subplot(122), plt.imshow(out, cmap='gray')
plt.title('Transformada de Fourier del Filtro Gaussiano'), plt.xticks([]), plt.yticks([])

plt.show()



In [ ]:
# visualizar con el código que nos otorgó el profe en Clase 3
plt.figure(figsize=(12,12))
plt.subplot(1,2,1)
plt.axis('off')
plt.title("Kernel Gausiano")
plt.imshow(gauss_filter, cmap='jet')
plt.subplot(1,2,2)
plt.axis('off')
plt.title("FFT del kernel")
plt.imshow(np.abs(gauss_filter_fft), cmap='jet')

## Ejercicio 2

2. Usando el método descripto en el paper “Image Sharpness Measure for Blurred Images in Frequency Domain” comparar el resultado de un filtrado por convolución con el filtrado espectral.



El algoritmo es el siguiente:

![alt text](https://www.dropbox.com/s/vifrrrtrwxmeep7/algoritmo1.png?dl=1)

In [ ]:
def image_quality_measure(imagen):

    assert imagen.size != 0 # check imagen no vacia
    assert imagen.shape[1] != 0  # check tiene columnas

    # Calcular la transformada de Fourier del kernel
    _fft = np.fft.fft2(imagen)

    # centrarla al origen
    _fft_shift = np.fft.fftshift(_fft)

    # calculo valor absoluto y me quedo con la maxima
    M = np.max(np.log(np.abs(_fft_shift)))

    T_h = (imagen > (M/1000)).sum()

    N = imagen.shape[1] # nro de columnas en la imagen

    que_regreso = T_h / (M * N )

    return que_regreso

In [ ]:
url = 'https://www.dropbox.com/s/8efkws4d0amkrgo/CoordCrom_1.png?dl=1'
#url = 'https://www.dropbox.com/s/8efkws4d0amkrgo/CoordCrom_2.png?dl=1'
#url = 'https://www.dropbox.com/s/8efkws4d0amkrgo/CoordCrom_3.png?dl=1'
myImg = io.imread(url)
imagen = cv.cvtColor(myImg, cv2.COLOR_BGR2GRAY) # cargo imagen robotito en escala de grises
#cv2_imshow(imagen)
plt.figure(figsize=(8,8))
plt.imshow(imagen, cmap='gray')

In [ ]:
# Parámetros de prueba
h = imagen.shape[0]
w = imagen.shape[1]
kernel_size = 15
sigma_value = 27


In [ ]:
# 1) imagen filtrada con gaussian blur
# 2) usar el metodo create_gaussian
# 3) aplicar

# Crear filtro gaussiano y su transformada de Fourier
gauss_filter, gauss_filter_fft = create_gauss_filter(h, w, kernel_size, sigma_value)



In [ ]:

filter_conv = cv2.filter2D(imagen, -1, gauss_filter)

img_fft = np.fft.fft2(imagen)

filtered = img_fft * gauss_filter_fft

#img_gauss = np.real(np.fft.ifft2(np.fft.ifftshift(filtered)))
img_gauss = np.fft.ifftshift(np.real(np.fft.ifft2(filtered)))


plt.figure(figsize=(12,12))
plt.subplot(1,2,1)
plt.axis('off')
plt.title("Kernel Gausiano")
plt.imshow(filter_conv, cmap='gray')

plt.subplot(1,2,2)
plt.imshow(img_gauss, cmap='gray')
plt.title('Desenfoque con fft')


In [ ]:
image_quality_measure(imagen)

In [ ]:
image_quality_measure(filter_conv)

In [ ]:

image_quality_measure(img_gauss)

## Ejercicio 3

3. Repetir la comparación usando uno de los métodos descriptos en el apéndice del paper “Analysis of focus measure operators in shape-from-focus”

Método A.24: gray-level variance (STA3)

$$ \phi_{x,y} = \sum_{i,j} (I(i,j)-μ)^2 $$



In [ ]:
def gray_level_variance(image):
  media = np.mean(image)
  r = image - media
  cuadrado = np.power(r, 2)
  return np.sum(cuadrado)

In [ ]:
gray_level_variance(imagen)

In [ ]:
gray_level_variance(filter_conv)

In [ ]:
gray_level_variance(img_gauss)

Repito las transformaciones para distintos valores de sigma para comparar ambas métricas

In [ ]:
# Parámetros de prueba
h = imagen.shape[0]
w = imagen.shape[1]
kernel_size = 15
sigma_value = [5, 8, 13, 21,34,55,89,144,233,377,610]

imq_gausiano = []
img_ft = []
glv_gausiano = []
glv_ft = []

for sig in sigma_value:
  gauss_filter, gauss_filter_fft = create_gauss_filter(h, w, kernel_size, sig)

  filter_conv = cv2.filter2D(imagen, -1, gauss_filter)

  img_fft = np.fft.fft2(imagen)

  filtered = img_fft * gauss_filter_fft

  img_gauss = np.fft.ifftshift(np.real(np.fft.ifft2(filtered)))

  imq_gausiano = np.append(imq_gausiano,image_quality_measure(filter_conv))
  img_ft = np.append(img_ft,image_quality_measure(img_gauss))
  glv_gausiano= np.append(glv_gausiano,gray_level_variance(filter_conv))
  glv_ft = np.append(glv_ft,gray_level_variance(img_gauss))


In [ ]:
imq_gausiano